In [1]:
#leer archivo de acepta------------------------------------------
import pandas as pd
import pyodbc 
import smtplib
from datetime import datetime
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.header import Header
from datetime import timedelta

path= "D:/Program Files/Python - Projects/test/test.xls"

df1 = pd.read_excel(path)
df1.drop(['receptor','razon_social_receptor','moneda','isc','ivap','operacionesexoneradas','operacionesinafectas','operacionesgratis',
         'descuentoglobal','otroscargos','otrostributos','icbper','resumen_boleta','uri','referencias','mensaje_sii','motivo_observacion']
        ,axis=1, inplace=True)

#df.isnull().any(axis=1).sum()
#condicion=df1[df1["nro_cpe"].isnull() | df1["igv"].isnull()] 
df_rechazado=df1[df1["estado_acepta"]=="RECHAZADO"] 



#print(df_rechazado)
#print(df_rechazado.head())

#print(df1.head(10))
#--------------------------------------------------------

#recuperar consulta de la BD-TFL- RMS---------------------------
#datos de conexion
server = '191.1.64.111' 
database = 'TFL' 
username = 'retailuser' 
password = 'retail'

#cadena de conexion
conn= pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER=191.1.64.111;DATABASE=TFL;UID=retailuser;PWD=retail')
cursor=conn.cursor()

#setear fecha ini/fin un dia anterior
fIni = datetime.now()- timedelta(days=1)
fIni=fIni.replace(hour=0,minute=0,second=0).strftime('%Y-%m-%d %H:%M:%S')
fFin = datetime.now()- timedelta(days=1)
fFin=fFin.replace(hour=23,minute=59,second=59).strftime('%Y-%m-%d %H:%M:%S')
#query para ejecutar sp 2023-04-11 00:00:00 2023-04-11 23:59:59
#query="EXEC Sp_Integration_StoreSummaryCustomerCompany_forDay '2023-04-11 00:00:00' , '2023-04-11 23:59:59' , 1"
query="EXEC Sp_Integration_StoreSummaryCustomerCompany_forDay @DateFrom = '{0}' ,@DateTo='{1}', @StoreNo={2}".format(fIni,fFin,9999)
#mostrar dataframe usando lib pandas
df2 = pd.read_sql_query(query,conn)
#print(df2.head(10))

conn.close()
#---------------------------------

# busca los nro_cpe de RMS que no estan en nro_cpe de Acepta
#df_RmsDocNotInAcep=df2[~df2.nro_cpe.isin(df1.nro_cpe)]

df_RmsDocNotInAcep = df2[~df2.nro_cpe.isin(df1.nro_cpe) & df2.nro_cpe.notna() & (df2.nro_cpe != '')]

df_AcepDocNotInRMS = df1[~df1.nro_cpe.isin(df2.nro_cpe) & df1.nro_cpe.notna() & (df1.nro_cpe != '')]

resultadoFinalAcep = pd.concat([df_rechazado,df_AcepDocNotInRMS])

#print(df_RmsDocNotInAcep)
#print(resultado.head())
#print(resultado.head(10))
#----------------------------------------------
#mostrar union : df_rechazado-->acepta y df_RmsDocNotInAcep--> resultado de buscar nro_cpe en Acepta(not isin)

#resultado_final = pd.concat([df_rechazado,df_RmsDocNotInAcep],axis=1) #inicial
resultado_finalAcepta =df_rechazado
resultado_finalRMS = df_RmsDocNotInAcep
#print(resultado_final)
#-------------------------------------------
#-----------------ENVIAR CORREO (ALERTA)-----------------------
html = """\
<html>
  <head></head>
  <body>
    <h3>ACEPTA</h3>
    {0}
    <br>
    <br>
    <h3>RMS</h3>
    {1}
  </body>
</html>
""".format(resultadoFinalAcep.to_html(index=False),df_RmsDocNotInAcep.to_html(index=False))

part1 = MIMEText(html, 'html')
message= part1
message['Subject'] = Header("Cruce RMS - Acepta",'utf-8')

server=smtplib.SMTP('191.1.64.16',25)
server.starttls()
server.login('dsalazar@topitop.com.pe','elr284SD')
destinatarios = ['dsalazar@topitop.com.pe','cbravo@topitop.com.pe']

server.sendmail('dsalazar@topitop.com.pe',destinatarios,message.as_string())
server.quit()
print("exito")








C:\Users\Diego Martin\AppData\Local\Temp\ipykernel_11964\1060407847.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql_query(query,conn)


exito


In [ ]:
import pyodbc  
import pandas as pd
from datetime import datetime
from datetime import timedelta

#datos de conexion
server = '191.1.64.111' 
database = 'TFL' 
username = 'retailuser' 
password = 'retail'

#cadena de conexion
conn= pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER=191.1.64.111;DATABASE=TFL;UID=retailuser;PWD=retail')
cursor=conn.cursor()

#setear fecha ini/fin un dia anterior
fIni = datetime.now()- timedelta(days=1)
fIni=fIni.replace(hour=0,minute=0,second=0).strftime('%Y-%m-%d %H:%M:%S')
fFin = datetime.now()- timedelta(days=1)
fFin=fFin.replace(hour=23,minute=59,second=59).strftime('%Y-%m-%d %H:%M:%S')
#query para ejecutar sp 2023-04-11 00:00:00 2023-04-11 23:59:59
#query="EXEC Sp_Integration_StoreSummaryCustomerCompany_forDay '2023-04-11 00:00:00' , '2023-04-11 23:59:59' , 1"
query="EXEC Sp_Integration_StoreSummaryCustomerCompany_forDay @DateFrom = '{0}' ,@DateTo='{1}', @StoreNo={2}".format(fIni,fFin,1)
#mostrar dataframe usando lib pandas
df = pd.read_sql_query(query,conn)
print(df.head(10))

conn.close()

In [29]:
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
import smtplib
import pandas as pd

staff_df = pd.DataFrame([ {"Name" : "Kelly" ,"Role" : "Director of HR","Alias":"kel"},
                          {"Name" : "Sally" ,"Role" : "Course liasion","Alias":"sal"},
                          {"Name" : "James" ,"Role" : "Grader","Alias":"jam"}])



student_df = pd.DataFrame([ {"Nombre" : "James" ,"Role" : "Grader","Date":"2023/01/02"},
                          {"Nombre" : "Micky" ,"Role" : "Law","Date":"2022/01/02"},
                          {"Nombre" : "Sally" ,"Role" : "Engineering","Date":"2020/01/02"}])

result=pd.concat([staff_df,student_df],axis=1)

#staff_df=staff_df.style.format_index(None)


html = """\
<html>
  <head></head>
  <body>
    <h3>ACEPTA</h3>
    {0}
    <br>
    <br>
    <h3>RMS</h3>
    {1}
  </body>
</html>
""".format(staff_df.to_html(index=False),student_df.to_html())

part1 = MIMEText(html, 'html')
message= part1
message['Subject'] = Header("Cruce RMS - Acepta",'utf-8')

server=smtplib.SMTP('191.1.64.16',25)
server.starttls()
server.login('dsalazar@topitop.com.pe','elr284SD')
destinatarios = ['dsalazar@topitop.com.pe']

server.sendmail('dsalazar@topitop.com.pe',destinatarios,message.as_string())
server.quit()
print("exito")






exito


In [9]:
from datetime import timedelta
from datetime import date

fecha= date.today() - timedelta(days=1)
fecha = fecha.strftime("%Y%m%d")
print(fecha)
#path = "\\191.1.64.110\VentasRMS"+"CruceAcepta_"+fecha
#print (path)

20230516


In [ ]:
import smtplib

message= "Hola mundo desde web"

server=smtplib.SMTP('smtp.gmail.com',465)
server.starttls()
server.login('dmsocl13@gmail.com','ggsdtddfgcljyqyd')
server.sendmail('dmsocl13@gmail.com','dmsocl13@gmail.com','message')
server.quit()
print("exito")

In [ ]:

import smtplib
import ssl
from email.message import EmailMessage

# Define email sender and receiver
email_sender = 'dsalazar@topitop.com.pe'
email_password = 'elr284SD'
email_receiver = 'dsalazar@topitop.com.pe'

# Set the subject and body of the email
subject = 'hola xd!'
body = """
hola mundo
"""

em = EmailMessage()
em['From'] ='dsalazar@topitop.com.pe'
em['To'] ='dsalazar@topitop.com.pe'
em['Subject'] = subject
em.set_content(body)

# Add SSL (layer of security)
context = ssl.create_default_context()

# Log in and send the email
with smtplib.SMTP_SSL('191.1.64.16', 25, context=context) as smtp:
    smtp.login(email_sender, email_password)
    smtp.sendmail(email_sender, email_receiver, em.as_string())

In [8]:
import smtplib

message= "test python"

server=smtplib.SMTP('191.1.64.16',25)
server.starttls()
server.login('dsalazar@topitop.com.pe','elr284SD')
server.sendmail('dsalazar@topitop.com.pe','cbravo@topitop.com.pe',message)
server.quit()
print("exito")

exito
